In [1]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import pprint
import json
import urllib
import requests

# Get Covid data from api.covidtracking.com
1. suppose 'data' is the retrieved json, then it has keys ['links', 'meta', 'data']<br>
2. For us/daily, data['data'] stores daily number of cases, with keys ['date', 'states', 'cases', 'testing', 'outcomes']<br>
['date'] is the date in YYYY-MM-DD<br>
['state'] is the number of states<br>
['cases'] has a single key ['total'] with ['value'] (raw number) and ['calculated'] (some statistics)<br>
['testing'] is structured the same way as ['cases']<br>
['outcomes']<br>
3. For state/daily, data['data'] has keys ['date', 'state', 'meta', 'cases', 'tests', 'outcomes']

In [106]:
def get_state_codes():
    '''
    Query api to get state metadata, incl state codes
    '''
    l = 'https://api.covidtracking.com/v2/states.json'
    with urllib.request.urlopen(l) as url:
        link_json = json.load(url)
    pd.DataFrame(link_json['data']).to_csv('./data/states.csv')

In [128]:
def retrieve_json(link):
    '''
    Given a link, query and save the json file. 
    Save all state file names in global variable covid_files.
    '''
    try:
        with urllib.request.urlopen(link) as url:
            link_json = json.load(url)
        filename = './data/' + link.split('v2/')[1].replace('/', '_')
        with open(filename, 'w') as f:
            json.dump(link_json, f)
        print('saved ' + filename)
        if filename != './data/us_daily.json':
            covid_files.append(filename)
    except:
        print('retrieval failed ' + link)

In [113]:
def get_all_json():
    '''
    Get json files of us/daily.json and all the states/{state}/daily/simple.json
    '''
    covidtracking_api = 'https://api.covidtracking.com/v2/'
    
    retrieve_json(covidtracking_api+'us/daily.json')
    
    states_codes = pd.read_csv('./data/states.csv')['state_code'].str.lower()
    for state in states_codes:
        retrieve_json(covidtracking_api + 'states/' + state + '/daily/simple.json')

In [169]:
def clean_df_from_json(filename):
    '''
    Given a json file, transform it to pandas DataFrame and save it locally as .csv.
    '''
    with open(filename, 'r') as f:
        temp = json.load(f)
    temp = pd.DataFrame(temp['data'])
    
    if filename != './data/us_daily.json': # parse state data
        temp['cases_total'] = temp['cases'].apply(lambda x: x['total'])
        temp['cases_confirmed'] = temp['cases'].apply(lambda x: x['confirmed'])
        temp['cases_probable'] = temp['cases'].apply(lambda x: x['probable'])
        temp['tests_total'] = temp['tests'].apply(lambda x: x['pcr']['specimens']['total'])
        temp['tests_positive'] = temp['tests'].apply(lambda x: x['pcr']['specimens']['positive'])
        temp = temp.drop(['cases', 'tests'], axis=1)
        temp.to_csv(filename.replace('json', 'csv'))
        print('saved ' + filename.replace('json', 'csv'))
    else: # parse US data
        temp['cases_total'] = temp['cases'].apply(lambda x: x['total']['value'])
        temp['cases_percent'] = temp['cases'].apply(lambda x: x['total']['calculated']['population_percent'])
        temp['cases_increase'] = temp['cases'].apply(lambda x: x['total']['calculated']['change_from_prior_day'])
        temp['cases_7d_change'] = temp['cases'].apply(lambda x: x['total']['calculated']['seven_day_change_percent'])
        temp['testing_total'] = temp['testing'].apply(lambda x: x['total']['value'])
        temp['testing_percent'] = temp['testing'].apply(lambda x: x['total']['calculated']['population_percent'])
        temp['testing_increase'] = temp['testing'].apply(lambda x: x['total']['calculated']['change_from_prior_day'])
        temp['testing_7d_change'] = temp['testing'].apply(lambda x: x['total']['calculated']['seven_day_change_percent'])
        temp = temp.drop(['cases', 'testing'], axis=1)
        temp.to_csv('./data/us_daily.csv')
        print('saved us_daily.csv')

In [170]:
def clean_all_df():
    '''
    Parse all US and state json files.
    '''
    clean_df_from_json('./data/us_daily.json')
    for file in covid_files:
        clean_df_from_json(file)

In [107]:
get_state_codes()

In [93]:
covid_files = [] # store state json file names
get_all_json()

saved states_al_daily_simple.json
saved states_ak_daily_simple.json
saved states_az_daily_simple.json
saved states_ar_daily_simple.json
saved states_ca_daily_simple.json
saved states_co_daily_simple.json
saved states_ct_daily_simple.json
saved states_de_daily_simple.json
saved states_dc_daily_simple.json
saved states_fl_daily_simple.json
saved states_ga_daily_simple.json
saved states_hi_daily_simple.json
saved states_id_daily_simple.json
saved states_il_daily_simple.json
saved states_in_daily_simple.json
saved states_ia_daily_simple.json
saved states_ks_daily_simple.json
saved states_ky_daily_simple.json
saved states_la_daily_simple.json
saved states_me_daily_simple.json
saved states_md_daily_simple.json
saved states_ma_daily_simple.json
saved states_mi_daily_simple.json
saved states_mn_daily_simple.json
saved states_ms_daily_simple.json
saved states_mo_daily_simple.json
saved states_mt_daily_simple.json
saved states_ne_daily_simple.json
saved states_nv_daily_simple.json
saved states_n

In [171]:
clean_all_df()

saved us_daily.csv
saved states_al_daily_simple.csv
saved states_ak_daily_simple.csv
saved states_az_daily_simple.csv
saved states_ar_daily_simple.csv
saved states_ca_daily_simple.csv
saved states_co_daily_simple.csv
saved states_ct_daily_simple.csv
saved states_de_daily_simple.csv
saved states_dc_daily_simple.csv
saved states_fl_daily_simple.csv
saved states_ga_daily_simple.csv
saved states_hi_daily_simple.csv
saved states_id_daily_simple.csv
saved states_il_daily_simple.csv
saved states_in_daily_simple.csv
saved states_ia_daily_simple.csv
saved states_ks_daily_simple.csv
saved states_ky_daily_simple.csv
saved states_la_daily_simple.csv
saved states_me_daily_simple.csv
saved states_md_daily_simple.csv
saved states_ma_daily_simple.csv
saved states_mi_daily_simple.csv
saved states_mn_daily_simple.csv
saved states_ms_daily_simple.csv
saved states_mo_daily_simple.csv
saved states_mt_daily_simple.csv
saved states_ne_daily_simple.csv
saved states_nv_daily_simple.csv
saved states_nh_daily_si

# Get Covid data (of March 10) from worldometers

In [6]:
wdm = 'https://www.worldometers.info/coronavirus/country/us/'
wdm_page = requests.get(wdm)

In [12]:
df = pd.read_html(wdm_page.content)[1]
df.columns.to_list() # check weird column names

['#',
 'USAState',
 'TotalCases',
 'NewCases',
 'TotalDeaths',
 'NewDeaths',
 'TotalRecovered',
 'ActiveCases',
 'Tot\xa0Cases/1M pop',
 'Deaths/1M pop',
 'TotalTests',
 'Tests/ 1M pop',
 'Population',
 'Projections']

In [16]:
# change weird column names
df = df.rename(columns={'Tot\xa0Cases/1M pop': 'TotCasesPer1MPop', 
                        'Deaths/1M pop':'DeathsPer1MPop', 'Tests/ 1M pop': 'TestsPer1MPop'})
df.to_csv('./data/us_0310.csv')

# Get weather data from api.weather.gov (abandoned)

Abandoned because the api does not provide enough data<br>
(only weather from the last 3 days are available)<br>
included here as a record of the data scraping process for completeness

In [100]:
# the county_stations is always empty as returned by the api, so we have another function
# get_state_stations
def get_state_counties(state):
    state_abbr = states[states['name']==state]['state_code'].values[0]
    t = f'https://api.weather.gov/zones?area={state_abbr}&type=county'
    with urllib.request.urlopen(t) as url:
            k = url.read()
    print(t + ' api call is successful')
    j = json.loads(k)['features']
    
    county_links = [item['id'] for item in j]
    county_ids = [item['properties']['id'] for item in j]
    county_names = [item['properties']['name'] for item in j]
    county_states = [item['properties']['state'] for item in j]
    county_stations = [item['properties']['observationStations'] for item in j] # empty
    
    df = pd.DataFrame({'link': county_links, 'id': county_ids, 'name': county_names,
                      'state_code': county_states, 'stations': county_stations})
    df['state'] = [state]*len(j)
    return df

In [23]:
# the api does not show any stations for a given county, so we have to get all stations in
# a state and match them to their counties
def get_state_stations(state):
    state_abbr = states[states['name']==state]['state_code'].values[0]
    t = f'https://api.weather.gov/stations?state={state_abbr}'
    with urllib.request.urlopen(t) as url:
            k = url.read()
    print(t + ' api call is successful')
    j = json.loads(k)['features']
    
    station_forecasts = []
    for i, item in enumerate(j):
        if 'forecast' in item['properties'].keys():
            station_forecasts.append(item['properties']['forecast'])
        else:
            station_forecasts.append(np.nan)

    station_counties = []
    for i, item in enumerate(j):
        if 'county' in item['properties'].keys():
            station_counties.append(item['properties']['county'].split('/')[-1])
        else:
            station_counties.append(np.nan)
    
    df = pd.DataFrame({'forecast': station_forecasts, 'county': station_counties})
    df = df.dropna(axis=0)
    df = df.drop_duplicates(ignore_index=True)
    
    return df

In [122]:
def get_station_county_pair(state):
    counties_of_state = get_state_counties(state)
    stations_of_state = get_state_stations(state)
    df = pd.merge(cali, cali_station, left_on='id', right_on='county', 
                  suffixes=['_county', '_station'])
    df = df.drop('county', axis=1)
    df.to_csv(f'./data/{state}_station_county_pair.csv')

In [131]:
states = pd.read_csv('./data/states.csv', index_col=0)

In [123]:
get_station_county_pair('California')

https://api.weather.gov/zones?area=CA&type=county api call is successful
https://api.weather.gov/stations?state=CA api call is successful


# Get weather data from NOAA
Get TAVG (average temperature) for counties from 2020-01-21 to 2021-03-08 using fsip through NOAA api

In [ ]:
# get all available datatypes of NOAA
all_datatypes_df = pd.DataFrame()
for i in ['asc', 'desc']: # query twice because the api return has a limit of 1000
    datatypes = f'https://www.ncdc.noaa.gov/cdo-web/api/v2/datatypes?limit=1000&sortorder=' + i
    datatypes_r = requests.get(datatypes, headers={'token': token})

    temp = json.loads(datatypes_r.text)['results']
    mindate = [item['mindate'] for item in temp]
    maxdate = [item['maxdate'] for item in temp]
    name = [item['name'] for item in temp]
    _id = [item['id'] for item in temp]
    datatype_df = pd.DataFrame({'mindate': mindate, 'maxdate': maxdate, 'name': name, 'id': _id})
    all_datatypes_df = pd.concat([all_datatypes_df, datatype_df])

all_datatypes_df.drop_duplicates(ignore_index=True, inplace=True)
all_datatypes_df = all_datatypes_df.sort_values('id')
all_datatypes_df.to_csv('noaa_datatypes.csv')

In [33]:
def get_county_weather(state):
    '''
    Given a state, get all the available TAVG data for its counties
    '''
    county_fips = us_counties[us_counties['state']==state]['fips'].unique()
    params1 = '&startdate=2020-01-21&enddate=2020-12-31&datatypeid=TAVG&limit=1000&units=standard'
    params2 = '&startdate=2021-01-01&enddate=2021-03-08&datatypeid=TAVG&limit=1000&units=standard'
    total = pd.DataFrame()
    print(f'{len(county_fips)} counties')
    
    for fips in county_fips:
        for params in [params1, params2]: # query twice because of the api return limit of 1k
            
            # make connection
            link = noaa_api + f'&locationid=FIPS:{int(fips):05}' + params
            try:
                r = requests.get(link, headers={'token': token}, timeout=5)
            except:
                print(fips, 'timeout', end=' ')
                continue
            if r.status_code != 200:
                print(fips, 'failed', end=' ')
                continue
            print(fips, 'connected', end=' ')
            
            # parse response
            if json.loads(r.text): # response can be empty
            
                d = json.loads(r.text)['results']
                dates = [item['date'] for item in d]
                values = [item['value'] for item in d]
                df = pd.DataFrame({'TAVG': values, 'date': dates})
                df['fips'] = [fips] * len(d)
                
                # there can be 2 TAVGs for one day
                # this step takes the mean of the 2 TAVGs
                mean_tavg = df.groupby(['fips', 'date'])['TAVG'].mean().values
                df = df.drop('TAVG', axis=1)
                df = df.drop_duplicates(ignore_index=True)
                df['mean_temp'] = mean_tavg

                total = pd.concat((total, df))
                
    total.to_csv(f'./weather_data/{state}_weather.csv')
    return total

In [3]:
us_counties = pd.read_csv('./data/us-counties.csv')
us_counties['date'] = pd.to_datetime(us_counties['date'])
us_counties = us_counties.dropna()

In [4]:
noaa_api = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND'
token = 'rilsfDWJGSwodWyqrUaOsREORAwwngJa' # my token

In [34]:
_ = get_county_weather('California')

58 counties
6059.0 connected 6059.0 connected 6037.0 connected 6037.0 connected 6085.0 connected 6085.0 connected 6075.0 connected 6075.0 connected 6073.0 connected 6073.0 connected 6023.0 connected 6023.0 connected 6067.0 connected 6067.0 connected 6095.0 connected 6095.0 connected 6041.0 connected 6041.0 connected 6055.0 connected 6055.0 connected 6097.0 connected 6097.0 connected 6001.0 connected 6001.0 connected 6061.0 connected 6061.0 connected 6081.0 connected 6081.0 connected 6013.0 connected 6013.0 connected 6113.0 connected 6113.0 connected 6019.0 connected 6019.0 connected 6039.0 connected 6039.0 connected 6065.0 connected 6065.0 connected 6087.0 connected 6087.0 connected 6089.0 connected 6089.0 connected 6077.0 connected 6077.0 connected 6111.0 timeout 6111.0 timeout 6099.0 connected 6099.0 connected 6107.0 connected 6107.0 connected 6069.0 timeout 6069.0 timeout 6079.0 timeout 6079.0 connected 6071.0 connected 6071.0 connected 6083.0 connected 6083.0 connected 6057.0 conne

,date,fips,mean_temp
0,2020-01-21T00:00:00,6059.0,51.5
1,2020-01-22T00:00:00,6059.0,53.0
2,2020-01-23T00:00:00,6059.0,62.5
3,2020-01-24T00:00:00,6059.0,62.0
4,2020-01-25T00:00:00,6059.0,57.5
...,...,...,...
62,2021-03-04T00:00:00,6049.0,39.1
63,2021-03-05T00:00:00,6049.0,42.7
64,2021-03-06T00:00:00,6049.0,31.3
65,2021-03-07T00:00:00,6049.0,33.2


In [35]:
_ = get_county_weather('Texas')

254 counties


48029.0 connected 48029.0 connected 48157.0 connected 48157.0 connected 48201.0 connected 48201.0 connected 48085.0 connected 48085.0 connected 48113.0 connected 48113.0 connected 48339.0 connected 48339.0 connected 48439.0 connected 48439.0 connected 48183.0 connected 48183.0 connected 48027.0 connected 48027.0 connected 48141.0 connected 48141.0 connected 48167.0 connected 48167.0 connected 48423.0 connected 48423.0 connected 48453.0 connected 48453.0 connected 48039.0 timeout 48039.0 connected 48209.0 connected 48209.0 connected 48121.0 connected 48121.0 connected 48037.0 connected 48037.0 connected 48321.0 connected 48321.0 connected 48041.0 connected 48041.0 connected 48139.0 connected 48139.0 connected 48303.0 connected 48303.0 connected 48285.0 connected 48285.0 connected 48309.0 connected 48309.0 connected 48325.0 timeout 48325.0 connected 48401.0 connected 48401.0 connected 48479.0 connected 48479.0 connected 48485.0 connected 48485.0 connected 48491.0 connected 48491.0 connec

48229.0 connected 48383.0 connected 48383.0 connected 48271.0 timeout 48271.0 connected 48235.0 timeout 48235.0 connected 48023.0 connected 48023.0 connected 48105.0 connected 48105.0 connected 48385.0 connected 48385.0 connected 48425.0 connected 48425.0 connected 48413.0 connected 48413.0 connected 48435.0 connected 48435.0 connected 48461.0 connected 48461.0 connected 48109.0 connected 48109.0 connected 48327.0 connected 48327.0 timeout 48443.0 connected 48443.0 connected 48137.0 connected 48137.0 connected 48243.0 connected 48243.0 connected 48261.0 connected 48261.0 timeout 48447.0 connected 48447.0 timeout 48155.0 connected 48155.0 connected 48263.0 connected 48263.0 connected 48311.0 connected 48311.0 connected 48433.0 connected 48433.0 connected 48431.0 connected 48431.0 connected 48033.0 connected 48033.0 connected 48269.0 timeout 48269.0 connected 48301.0 connected 48301.0 connected 

In [40]:
_ = get_county_weather('Florida')

67 counties
12057.0 connected 12057.0 connected 12081.0 connected 12081.0 connected 12113.0 connected 12113.0 connected 12011.0 connected 12011.0 connected 12071.0 connected 12071.0 connected 12015.0 connected 12015.0 connected 12091.0 connected 12091.0 connected 12127.0 connected 12127.0 connected 12001.0 connected 12001.0 connected 12021.0 connected 12021.0 connected 12089.0 connected 12089.0 connected 12101.0 connected 12101.0 connected 12103.0 connected 12103.0 connected 12086.0 connected 12086.0 connected 12109.0 connected 12109.0 connected 12019.0 connected 12019.0 connected 12031.0 timeout 12031.0 connected 12099.0 timeout 12099.0 connected 12115.0 connected 12115.0 connected 12117.0 connected 12117.0 connected 12095.0 connected 12095.0 timeout 12097.0 connected 12097.0 connected 12003.0 connected 12003.0 connected 12017.0 connected 12017.0 connected 12033.0 connected 12033.0 connected 12069.0 connected 12069.0 connected 12009.0 connected 12009.0 connected 12105.0 connected 1210

In [41]:
_ = get_county_weather('North Dakota')

53 counties
38101.0 connected 38101.0 connected 38015.0 connected 38015.0 connected 38017.0 connected 38017.0 connected 38059.0 connected 38059.0 connected 38069.0 connected 38069.0 connected 38071.0 connected 38071.0 connected 38099.0 connected 38099.0 timeout 38025.0 connected 38025.0 connected 38031.0 connected 38031.0 connected 38089.0 connected 38089.0 connected 38049.0 connected 38049.0 connected 38051.0 connected 38051.0 connected 38003.0 connected 38003.0 connected 38023.0 connected 38023.0 connected 38055.0 connected 38055.0 connected 38061.0 connected 38061.0 connected 38085.0 connected 38085.0 connected 38035.0 connected 38035.0 connected 38065.0 connected 38065.0 connected 38037.0 connected 38037.0 timeout 38053.0 timeout 38053.0 connected 38105.0 connected 38105.0 connected 38013.0 timeout 38013.0 connected 38027.0 connected 38027.0 connected 38087.0 connected 38087.0 connected 38029.0 timeout 38029.0 connected 38011.0 timeout 38011.0 timeout 38093.0 connected 38093.0 conn

In [42]:
_ = get_county_weather('South Dakota')

66 counties
46005.0 timeout 46005.0 connected 46023.0 connected 46023.0 connected 46035.0 connected 46035.0 connected 46099.0 connected 46099.0 connected 46009.0 connected 46009.0 connected 46087.0 connected 46087.0 connected 46013.0 connected 46013.0 connected 46029.0 connected 46029.0 connected 46065.0 connected 46065.0 connected 46085.0 connected 46085.0 connected 46103.0 connected 46103.0 connected 46011.0 connected 46011.0 connected 46083.0 connected 46083.0 connected 46093.0 connected 46093.0 connected 46003.0 connected 46003.0 connected 46049.0 connected 46049.0 connected 46081.0 connected 46081.0 connected 46121.0 timeout 46121.0 timeout 46127.0 connected 46127.0 connected 46039.0 connected 46039.0 connected 46057.0 connected 46057.0 connected 46067.0 connected 46067.0 connected 46135.0 timeout 46135.0 connected 46025.0 connected 46025.0 connected 46027.0 connected 46027.0 connected 46047.0 connected 46047.0 connected 46109.0 timeout 46109.0 connected 46125.0 connected 46125.0 

In [43]:
_ = get_county_weather('Rhode Island')

5 counties
44001.0 connected 44001.0 connected 44003.0 connected 44003.0 connected 44005.0 timeout 44005.0 timeout 44007.0 connected 44007.0 connected 44009.0 connected 44009.0 connected 

In [47]:
_ = get_county_weather('New York')

57 counties
36119.0 connected 36119.0 connected 36059.0 connected 36059.0 connected 36087.0 connected 36087.0 connected 36091.0 connected 36091.0 connected 36103.0 connected 36103.0 connected 36111.0 connected 36111.0 connected 36055.0 connected 36055.0 connected 36001.0 connected 36001.0 connected 36025.0 timeout 36025.0 connected 36027.0 connected 36027.0 timeout 36043.0 connected 36043.0 connected 36071.0 connected 36071.0 connected 36093.0 timeout 36093.0 connected 36007.0 connected 36007.0 connected 36107.0 connected 36107.0 connected 36109.0 connected 36109.0 connected 36029.0 connected 36029.0 connected 36039.0 connected 36039.0 connected 36057.0 connected 36057.0 connected 36079.0 connected 36079.0 connected 36003.0 connected 36003.0 connected 36067.0 connected 36067.0 connected 36069.0 connected 36069.0 connected 36019.0 connected 36019.0 connected 36083.0 connected 36083.0 connected 36105.0 connected 36105.0 connected 36121.0 connected 36121.0 connected 36017.0 connected 3601

In [48]:
len(pd.read_csv('./weather_data/New York_weather.csv')['fips'].unique())

12